In [1]:
type Name =String
type Age = Integer
data Person = Person Name Age deriving Show

In [ ]:
data PersonInvalid = NameEmpty 
                    | AgeTooLow
                    deriving (Eq,Show)


In [7]:
notThe :: String -> Maybe String
notThe s
    | s == "the" = Nothing
    | otherwise = Just s
replaceThe :: String ->String
replaceThe s = unwords.(map (f.notThe)) $ words s
    where 
        f (Just s) = s
        f _ = "a"
replaceThe "the cow loves us"

Line 6: Redundant bracket
Found:
unwords . (map (f . notThe))
Why not:
unwords . map (f . notThe)

"a cow loves us"

In [14]:
vowels = "aeiou"
countTheBeforeVowel :: String -> Integer
countTheBeforeVowel s =  f (words s) 0
    where 
        isVowelInitial w = head w `elem` vowels
        f (prev:cur:rest) n = if isVowelInitial prev then f (cur:rest) (n+1) else f (cur:rest) n
        f _ n = n

countTheBeforeVowel "the evil cow"


1

In [18]:
countVowels :: String -> Int

countVowels s = go s 0
    where 
        go [] n = n
        go (x:xs) n= if x `elem` "aeiou" then go xs (n+1) else go xs n
-- countVowels "Mikolajczak"
countConsonants word = (length word) - (countVowels word)


Line 8: Redundant bracket
Found:
(length word) - (countVowels word)
Why not:
length word - (countVowels word)Line 8: Redundant bracket
Found:
(length word) - (countVowels word)
Why not:
(length word) - countVowels word

In [20]:
newtype Word' = Word' String deriving (Eq,Show)
vowels = "aeiou"
mkWord :: String -> Maybe Word'
mkWord word =  if (countVowels word) > (countConsonants word) then Nothing else Just (Word' word)

Line 4: Redundant bracket
Found:
(countVowels word) > (countConsonants word)
Why not:
countVowels word > (countConsonants word)Line 4: Redundant bracket
Found:
(countVowels word) > (countConsonants word)
Why not:
(countVowels word) > countConsonants word

In [33]:
data Nat = 
    Zero 
    |Succ Nat
    deriving (Eq,Show)
natToInteger :: Nat -> Integer
natToInteger Zero = 0
natToInteger (Succ n) = succ (natToInteger n)

integerToNat :: Integer -> Maybe Nat
integerToNat n
    | n < 0 = Nothing
    | otherwise = Just (integerToNat' n)
    where 
        integerToNat' 0 =  Zero
        integerToNat' n = Succ (integerToNat' (n-1))
integerToNat (1)

Line 16: Redundant bracket
Found:
(1)
Why not:
1

Just (Succ Zero)

## Small library for Maybe

In [36]:
isJust :: Maybe a -> Bool
isJust (Just _) = True
isJust _ = False

isJust (Just 1)
isJust Nothing

isNothing Nothing  = True
isNothing _ = False
isNothing Nothing

True

False

True

In [38]:
mayybee :: b -> (a->b) ->Maybe a -> b
mayybee b _ Nothing = b
mayybee b f (Just a) = f a
mayybee 0 (+1) Nothing
mayybee 0 (+1) (Just 1)

fromMaybe :: a-> Maybe a -> a
fromMaybe a Nothing = a
fromMaybe _ (Just a) = a
fromMaybe 0 Nothing
fromMaybe 0 (Just 1)

0

2

0

1

In [45]:
unJust (Just a) = a
unJust _ = error "Nothing after filter"
catMaybes :: [Maybe a] -> [a]
catMaybes maybes = map unJust (filter isJust maybes)
-- catMaybes [Just 1, Nothing ,Just 2]

flipMaybe :: [Maybe a] -> Maybe [a]
flipMaybe maybes = if all isJust maybes then Just (map unJust maybes) else Nothing
flipMaybe [Just 1,Nothing,Just 3]


Nothing

In [40]:
:t filter

filter :: forall a. (a -> Bool) -> [a] -> [a]

In [51]:
myIterate :: (a->a) -> a -> [a]
myIterate f a = a : myIterate f (f a)
take 3 $ myIterate (+ 1) 0

[0,1,2]

In [56]:
myUnfoldr :: (b->Maybe (a,b)) -> b -> [a]
myUnfoldr f b =  case f b of
        Nothing -> []
        Just (a,b) -> a: myUnfoldr f b
-- take 10 $ myUnfoldr (\b -> Just (b, b+1)) 0
betterIterate :: (a->a)->a->[a]
betterIterate f x = myUnfoldr (\b->Just (b, f b)) x
take 10 $ betterIterate (+1) 0

Line 7: Eta reduce
Found:
betterIterate f x = myUnfoldr (\ b -> Just (b, f b)) x
Why not:
betterIterate f = myUnfoldr (\ b -> Just (b, f b))

[0,1,2,3,4,5,6,7,8,9]

In [10]:
data BinaryTree a =
        Leaf
        | Node (BinaryTree a) a (BinaryTree a)
        deriving (Eq,Ord,Show)

unfold :: (a-> Maybe(a,b,a))-> a -> BinaryTree b
-- unfold f a = Leaf
unfold f tree = case f tree of 
        Nothing -> Leaf
        Just (left,newVal,right) -> Node (unfold f left) newVal  (unfold f right)
treeBuild :: Integer -> BinaryTree Integer
treeBuild n = unfold f 0
    where f m
            | m >= n = Nothing
            | otherwise = Just (m+1,m,m+1)
treeBuild 2

Node (Node Leaf 1 Leaf) 0 (Node Leaf 1 Leaf)